In [ ]:
# import libraries
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import csv
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score, f1_score
import pickle
import zipfile
import io
import os.path

import tensorflow as tf
print("tf.__version__ =", tf.__version__)
from tensorflow.keras.layers import Masking, Embedding, Bidirectional, LSTM
from tensorflow.keras.layers import Input, Flatten, Dense, TimeDistributed, AveragePooling1D, Activation, Dropout, Concatenate
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

import matplotlib
from matplotlib import pyplot
%matplotlib inline
from IPython import display
display.set_matplotlib_formats('svg')

aa_list = ['_PAD',
           'A',
           'R',
           'N',
           'D',
           'C',
           'E',
           'Q',
           'G',
           'H',
           'I',
           'L',
           'K',
           'M',
           'F',
           'P',
           'S',
           'T',
           'W',
           'Y',
           'V',
          ]
vocab_size = len(aa_list)
aa2index = {}
index2aa = {}
for index, aa in enumerate(aa_list):
    aa2index[aa] = index
    index2aa[index] = aa

IEDB_response_code = {'Positive': 1,
                      'Positive-High': 1,
                      'Positive-Intermediate': 1,
                      'Positive-Low': 1,
                      'Negative': 0,
                     }
MAX_LEN = 15

In [ ]:
# [need input] define T cell tables, running mode, patient, hla_alleles
# comment or uncomment the lines below to select the dataset of interest
# the default dataset is mel15

# use this csv file for human data
# tcell_table_file = "data.training/tcell/iedb.assay_tcell.hla_1.host_human/tcell_table_export_1578821658.csv"
# use this csv file for mouse data
tcell_table_file = "data.training/tcell/iedb.assay_tcell.hla_1.host_mouse/tcell_table_export_1640227737.csv"

# mode = 'real application' # in this mode, there is no data for evaluation
# patient = "melS2"
# hla_alleles = ["HLA-A*23:01", "HLA-A*24:02", "HLA-B*81:01", "HLA-B*40:01", "HLA-C*03:04", "HLA-C*07:02"]

mode = 'evaluation' # this mode is for the datasets in the DeepImmun manuscript
patient = "mel15"
hla_alleles = ["HLA-A*03:01", "HLA-A*68:01", "HLA-B*27:05", "HLA-B*35:03", "HLA-C*02:02", "HLA-C*04:01"]
# patient = "mel0D5P"
# hla_alleles = ["HLA-A*01:01", "HLA-A*23:01", "HLA-B*07:02", "HLA-B*15:01", "HLA-C*12:03", "HLA-C*14:02"]
# patient = "mel51"
# hla_alleles = ["HLA-A*01:01", "HLA-A*02:01", "HLA-B*14:02", "HLA-B*15:01", "HLA-C*03:04", "HLA-C*08:02"]
# patient = "mouseEL4"
# hla_alleles = ["H2-Db", "H2-Kb"]

print("patient =", patient)
print("hla_alleles =", hla_alleles)
print()


In [ ]:
##########################################################
##########################################################
##########################################################
# The code below is organized in this order:
#   TRAINING, PREDICTION, EVALUATION
# If you only want to do prediction using pretrained models,
# you can skip the TRAINING and jump to the PREDICTION
##########################################################
##########################################################
##########################################################

In [ ]:
##########################################################
##########################################################
##########################################################
# TRAINING
##########################################################
##########################################################
##########################################################

In [ ]:
# read IEDB tables; select HLA alleles, linear peptides; and prepare training data
output_file = "training_data_copy.csv"

def prepare_training_data(patient, hla_alleles):

  # read tcell assays table
  assay_dict = {}
  with open(tcell_table_file, 'r') as input_handle:
    csv_reader = csv.reader(input_handle, delimiter=',')
    header_1 = next(csv_reader)
    header_2 = next(csv_reader)
    header_list = []
    for x, y in zip(header_1, header_2):
      header_list.append(':'.join([x, y]))
    for row in csv_reader:
      assert len(row) == len(header_list)
      assay = {}
      for x, y in zip (header_list, row):
        assay[x] = y
      assay_id = assay['Reference:T Cell ID']
      assay_dict[assay_id] = assay
  print("len(assay_dict) =", len(assay_dict))
  
  print("patient =", patient)
  print("hla_alleles =", hla_alleles)
  assay_filtered = []
  for assay in assay_dict.values():
    allele = assay['MHC:Allele Name']
    epitope_type = assay['Epitope:Object Type']
    if (allele in hla_alleles and epitope_type == "Linear peptide"):
        assay_filtered.append(assay)

  print("len(assay_filtered) =", len(assay_filtered))
  peptide_set = set([x['Epitope:Description'] for x in assay_filtered])
  print("len(peptide_set) =", len(peptide_set))
  print()
  
  
  # output temporary csv
  with open(output_file, 'w') as output_handle:
    fieldnames = ['Reference:T Cell ID',
                  'MHC:Allele Name',
                  'Epitope:Epitope ID',
                  'Epitope:Object Type',
                  'Epitope:Description',
                  'Assay:Qualitative Measure',
                 ]
    csv_writer = csv.DictWriter(output_handle, fieldnames, delimiter=',')
    csv_writer.writeheader()
    for assay in assay_filtered:
      assay_subset = {k:v for k, v in assay.items() if k in fieldnames}
      csv_writer.writerow(assay_subset)

prepare_training_data(patient, hla_alleles)

In [ ]:
# read training data
training_data_file = "training_data_copy.csv"
training_pair_list = []
peptides_with_unknown_aa = 0
allele_freq_dict = {}
with open(training_data_file, 'r') as input_handle:
    csv_reader = csv.DictReader(input_handle, delimiter=',')
    for row in csv_reader:
        peptide = row['Epitope:Description']
        peptide = list(peptide)
        peptide_assertion = True
        for aa in peptide:
            if aa not in aa_list:
                peptide_assertion = False
                break
        if not peptide_assertion:
            peptides_with_unknown_aa += 1
            continue
        response = IEDB_response_code[row['Assay:Qualitative Measure']]
        allele = row['MHC:Allele Name']
        if allele in allele_freq_dict:
            allele_freq_dict[allele] += 1
        else:
            allele_freq_dict[allele] = 1
        training_pair_list.append([peptide, response])

print("allele_freq_dict =", allele_freq_dict)
print("peptides_with_unknown_aa = ", peptides_with_unknown_aa)
print("len(training_pair_list) = ", len(training_pair_list))
print("  positive = ", len([y for x, y in training_pair_list if y == 1]))
print("  negative = ", len([y for x, y in training_pair_list if y == 0]))
print("training_pair_list[0] = ", training_pair_list[0])


In [ ]:
# [need input] add self peptides from normal_hla.txt files
print("len(training_pair_list) ", len(training_pair_list))
normal_hla_file = patient + "_normal_hla.txt" # "mel15_normal_hla.txt"
print("normal_hla_file =", normal_hla_file)
with open(normal_hla_file, 'r') as input_handle:
    normal_hla = input_handle.readlines()
    normal_hla = [x.strip() for x in normal_hla]
normal_hla_neg = [[list(x), 0] for x in normal_hla]
print("len(normal_hla_neg) =", len(normal_hla_neg))
print("normal_hla_neg[0] =", normal_hla_neg[0])

training_pair_list_pos = [[x, y] for x, y in training_pair_list if y == 1]
print("len(training_pair_list_pos) ", len(training_pair_list_pos))
training_pair_list = training_pair_list_pos + normal_hla_neg
print("len(training_pair_list) ", len(training_pair_list))
print()


In [ ]:
# [need input] exclude mutated_test_set under evaluation mode
print("mode =", mode)
if mode == 'evaluation':
    print("len(training_pair_list) =", len(training_pair_list))
    mutated_test_file = "test." + patient + ".csv" # "test.mel15.csv"
    print("mutated_test_file =", mutated_test_file)
    mutated_test_list = set()
    if mutated_test_file[-3:] == 'csv':
        with open(mutated_test_file, 'r') as csv_handle:
            csv_reader = csv.DictReader(csv_handle)
            mutated_test_list = [row['peptide'] for row in csv_reader]
    elif mutated_test_file[-3:] == 'txt':
        with open(mutated_test_file, 'r') as file:
            mutated_test_list= [x.strip() for x in file.readlines()]
    print("len(mutated_test_list) =", len(mutated_test_list))
    training_overlap_mutated_test = [''.join(x[0]) for x in training_pair_list if ''.join(x[0]) in mutated_test_list]
    print("len(training_overlap_mutated_test) =", len(training_overlap_mutated_test))
    training_overlap_mutated_test = {x:training_overlap_mutated_test.count(x) for x in set(training_overlap_mutated_test)}
    print("training_overlap_mutated_test =", training_overlap_mutated_test)
    print("Exclude training_overlap_mutated_test")
    training_pair_list = [x for x in training_pair_list if ''.join(x[0]) not in mutated_test_list]
    print("len(training_pair_list) =", len(training_pair_list))
    print()


In [ ]:
# reduce training data to peptides rather than assays, filter out overlapping pos/neg peptides, filter length_8_14
print("len(training_pair_list) = ", len(training_pair_list))
peptide_set = set([''.join(x) for x, y in training_pair_list])
peptide_set_pos = set([''.join(x) for x, y in training_pair_list if y == 1])
peptide_set_neg = set([''.join(x) for x, y in training_pair_list if y == 0])
peptide_overlap = peptide_set_pos.intersection(peptide_set_neg)
print("len(peptide_set) = ", len(peptide_set))
print("  len(peptide_set_pos) = ", len(peptide_set_pos))
print("  len(peptide_set_neg) = ", len(peptide_set_neg))
print("  len(peptide_overlap) = ", len(peptide_overlap))
print("peptide_overlap =", peptide_overlap)
# convert set to sorted list to maintain the abc order
peptide_set_pos = sorted(list(peptide_set_pos))
peptide_set_neg = sorted(list(peptide_set_neg))
training_pair_list = [[list(x), 1] for x in peptide_set_pos] + [[list(x), 0] for x in peptide_set_neg]
training_pair_list = [[x, y] for x, y in training_pair_list if ''.join(x) not in peptide_overlap or y == 1]
print("len(training_pair_list) = ", len(training_pair_list))
print("  positive = ", len([y for x, y in training_pair_list if y == 1]))
print("  negative = ", len([y for x, y in training_pair_list if y == 0]))
print("training_pair_list[0] = ", training_pair_list[0])
print()

print("Filter length_8_14")
training_pair_list = [[x, y] for x, y in training_pair_list if len(x) >= 8 and len(x) <= 14]
print("len(training_pair_list) = ", len(training_pair_list))
print("training_pair_list[0] = ", training_pair_list[0])
print()


In [ ]:
# [need input] split train/valid/test and upsampling/downsampling
print("len(training_pair_list) = ", len(training_pair_list))
train_valid_set, test_set = train_test_split(training_pair_list, test_size=0.1, random_state=99)
train_set, valid_set = train_test_split(train_valid_set, test_size=0.1, random_state=99)
print("len(train_set) = ", len(train_set))
print("len(valid_set) = ", len(valid_set))
print("len(test_set) = ", len(test_set))
print()

# save the test_set for future evaluation
# test_set_file = "test_set." + patient + ".csv" # "test_set.mel15.csv"
# with open(test_set_file, 'w') as file:
#     csv_writer = csv.DictWriter(file, fieldnames=['peptide', 'response'])
#     csv_writer.writeheader()
#     for peptide, response in test_set:
#         peptide = ''.join(peptide)
#         response = 'positive' if response else 'negative'
#         csv_writer.writerow({'peptide': peptide, 'response': response})
# print("test_set_file =", test_set_file)
# print()

train_set_neg = [x for x in train_set if x[1] == 0]
train_set_pos = [x for x in train_set if x[1] == 1]
print("len(train_set_neg) =", len(train_set_neg))
print("len(train_set_pos) =", len(train_set_pos))
print()

def prepare_tensor(training_set):
    x_peptide = [x for x, y in training_set]
    y_tensor = np.array([y for x, y in training_set])
    x_tensor = [[aa2index[aa] for aa in peptide] for peptide in x_peptide]
    x_tensor = tf.keras.preprocessing.sequence.pad_sequences(
      x_tensor, 
      maxlen=MAX_LEN,
      dtype='int32',
      padding='post',
      value=0)
    return x_tensor, y_tensor

# x_train, y_train = prepare_tensor(train_set)
x_valid, y_valid = prepare_tensor(valid_set)
x_test, y_test = prepare_tensor(test_set)


In [ ]:
# [need input] model training
def train_model(x_train, y_train, x_valid, y_valid, model_path, num_epochs):
#     print("".join(["="] * 80)) # section-separating line
#     print("train_model()")

    # Model Training
    model = Sequential()
    model.add(Embedding(input_dim=vocab_size, output_dim=8, mask_zero=True))
    model.add(Bidirectional(LSTM(8, recurrent_initializer='glorot_uniform')))
    model.add(Dense(1, kernel_regularizer=regularizers.l2(0.01)))
    # dropout causes fluctuations ??
    #model.add(Dropout(0.5))
    #model.add(Dense(1))
    model.add(Activation('sigmoid'))
    #print(model.summary())
    #model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['binary_accuracy'])
    model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=[tf.keras.metrics.AUC(name='auc')])
    model_checkpoint = ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True)
    history = model.fit(x_train, y_train,
                        batch_size=32,
                        epochs=num_epochs,
                        validation_data=(x_valid, y_valid),
                        verbose=0,
                        callbacks=[model_checkpoint])
    
#     fig, ax = pyplot.subplots(1, 2)
#     ax[0].plot(history.history['loss'])
#     ax[0].plot(history.history['val_loss'])
#     ax[0].set_ylabel('loss')
#     ax[0].set_xlabel('epoch')
#     ax[0].legend(['loss', 'val_loss'], loc='upper left')
#     ax[1].plot(history.history['auc'])
#     ax[1].plot(history.history['val_auc'])
#     ax[1].set_ylabel('auc')
#     ax[1].set_xlabel('epoch')
#     ax[1].legend(['auc', 'val_auc'], loc='upper left')
    
    return model

# model training
print("model training")
model_name = patient + "_copy"
model_number = 100
model_paths = [model_name + "/" + "model" + "_" + model_name + "_" + str(m) + ".h5" for m in range(model_number)]
model_score_path = model_name + "/" + "model_score" + ".pkl"
print("len(model_paths) =", len(model_paths))
print("model_paths[0] =", model_paths[0])
print("model_score_path =", model_score_path)
print()
num_epochs = 100
for i, path in enumerate(model_paths):
    
    if os.path.isfile(path):
        continue
    
    print("path =", path)
    
    random.seed(99 + i)
    
#     # no sampling
#     train_set = train_set_neg + train_set_pos

    # downsampling negative samples
    train_set_neg_down = random.sample(train_set_neg, k=len(train_set_pos))
    train_set = train_set_neg_down + train_set_pos

#     # downsampling positive samples
#     train_set_pos_down = random.sample(train_set_pos, k=len(train_set_neg))
#     train_set = train_set_pos_down + train_set_neg
    
    random.shuffle(train_set)
    x_train, y_train = prepare_tensor(train_set)
    
    # model training
    model = train_model(x_train, y_train, x_valid, y_valid, path, num_epochs)


In [ ]:
# make predictions on valid set and save the results for model selection
if not os.path.isfile(model_score_path):
    print("make predictions on valid set and save the results for model selection")
    x_testing, y_testing = prepare_tensor(valid_set)
#     print("combine valid + test for more accurate model selection")
#     x_testing, y_testing = prepare_tensor(valid_set + test_set)
    print("len(y_testing) =", len(y_testing))
    print()
    y_pred_list = []
    for i, path in enumerate(model_paths):
        model = load_model(path)
        y_pred = model.predict(x_testing).flatten()
        y_pred_list.append(y_pred)
    model_score = [y_testing, y_pred_list]
    with open(model_score_path, 'wb') as file:
        pickle.dump(model_score, file)


In [ ]:
##########################################################
##########################################################
##########################################################
# PREDICTION
##########################################################
##########################################################
##########################################################

In [ ]:
# [need input] model selection
# before running this code, please add "_copy" to the model folder name, e.g. "mel15_copy"

print("model selection")
model_name = patient + "_copy"
model_score_path = model_name + "/" + "model_score" + ".pkl"
print("model_score_path =", model_score_path)
with open(model_score_path, 'rb') as file:
    model_score = pickle.load(file) # y_testing, y_pred_list
y_testing, y_pred_list = model_score
print("len(y_pred_list) =", len(y_pred_list))
print()

print("AUC of the average of the best models")
testing_auc_list = [roc_auc_score(y_testing, y_pred) for y_pred in y_pred_list]
pyplot.boxplot(testing_auc_list)
sorted_auc_indices = sorted(range(len(testing_auc_list)), key=lambda k: -testing_auc_list[k])
for best in [1, 10, 20, 40, 60, 80, len(testing_auc_list)]:
    y_pred_avg = np.mean([y_pred_list[i] for i in sorted_auc_indices[:best]], axis=0)
    y_pred_avg_auc = roc_auc_score(y_testing, y_pred_avg)
    print(best, "best models", y_pred_avg_auc)
print()


In [ ]:
# plot the score distribution of the average of the best models
print("plot the score distribution of the average of the best models")
best = 10
print(best, "best models")
y_score =  np.mean([y_pred_list[i] for i in sorted_auc_indices[:best]], axis=0)
fig, ax = pyplot.subplots(1, 2)
fpr, tpr, _ = roc_curve(y_testing, y_score)
auc = roc_auc_score(y_testing, y_score)
ax[0].plot(fpr,tpr,label="data 1, auc={:.2f}".format(auc))
ax[0].set_ylabel('True Positive Rate')
ax[0].set_xlabel('False Positive Rate')
ax[0].legend(loc=4)

y_score_0 = [b for a, b in zip(y_testing, y_score) if a == 0]
y_score_1 = [b for a, b in zip(y_testing, y_score) if a == 1]
my_dict = {'neg': y_score_0, 'pos': y_score_1}
ax[1].boxplot(my_dict.values())
ax[1].set_xticklabels(my_dict.keys())
pd_0 = pd.DataFrame({'neg': y_score_0})
pd_1 = pd.DataFrame({'pos': y_score_1})
pd.concat([pd_0, pd_1], ignore_index=True, axis=1).describe()


In [ ]:
# prediction

# function to draw random peptides of length 8-14 aa from swissprot.human.2020_06_15.fasta
fasta_path = "data.fasta/swissprot.human.2020_06_15.fasta"
print("fasta_path =", fasta_path)
num_proteins = 0
super_protein = ''
for record in SeqIO.parse(fasta_path, "fasta"):
    super_protein += str(record.seq)
    num_proteins += 1
print("num_proteins = ", num_proteins)
print("len(super_protein) = ", len(super_protein))
print()

def draw_random_peptides(num_peptides, super_protein):
    print("draw_random_peptides()")
    print("num_peptides =", num_peptides)
    super_protein_len = len(super_protein)
    random.seed(99)
    start_positions = random.choices(range(super_protein_len), k=num_peptides)
    lengths = random.choices(range(8, 14), k=num_peptides)
    random_peptide_list = [super_protein[s:s+l] for s,l in zip(start_positions, lengths)]
    random_peptide_list = sorted(set(random_peptide_list))
    random_peptide_list = [x for x in random_peptide_list if all([aa in aa_list for aa in x])]
    print("len(random_peptide_list) =", len(random_peptide_list))
    print()
    return random_peptide_list
# random_peptide_list = draw_random_peptides(10, super_protein)
# print(random_peptide_list)
# print()

def predict_sub(model, peptide_list):
    x_tensor = [[aa2index[aa] for aa in peptide] for peptide in peptide_list]
    x_tensor = tf.keras.preprocessing.sequence.pad_sequences(
      x_tensor, 
      maxlen=MAX_LEN,
      dtype='int32',
      padding='post',
      value=0)
    y_pred = model.predict(x_tensor).flatten()
    return y_pred

def predict(model_list, model_score, input_file, output_file):
#     print("predict()")
#     print("input_file = ", input_file)
#     print("output_file = ", output_file)

    if input_file[-3:] == 'csv':
        with open(input_file, 'r') as input_handle:
            csv_reader = csv.DictReader(input_handle, delimiter=',')
            csv_fieldnames = csv_reader.fieldnames
            csv_records = list(csv_reader)
    elif input_file[-3:] == 'txt':
        with open(input_file, 'r') as file:
            csv_fieldnames = ['peptide']
            csv_records = [{'peptide':x.strip()} for x in file.readlines()]
    print("number of input peptides =", len(csv_records))
    print()

    y_pred_list = []
    for model in model_list:
        y_pred = predict_sub(model, [record['peptide'] for record in csv_records])
        y_pred_list.append(y_pred)
    y_pred = np.mean(y_pred_list, axis=0)
    y_pred_std = np.std(y_pred_list, axis=0)

    random_peptide_list = draw_random_peptides(10000, super_protein)
    y_random_list = []
    for model in model_list:
        y_random = predict_sub(model, random_peptide_list)
        y_random_list.append(y_random)
    y_random = np.mean(y_random_list, axis=0)

    model_y_test, model_y_pred = model_score
    model_y_pred_0 = [b for (a, b) in zip(model_y_test, model_y_pred) if a == 0]
    model_y_pred_1 = [b for (a, b) in zip(model_y_test, model_y_pred) if a == 1]
    with open(output_file, 'w') as output_handle:
        csv_fieldnames += ['dpImmun', 'dpImmun_std', 'dpImmun_neg_pct', 'dpImmun_pos_pct', 'dpImmun_10k_pct']
        csv_writer = csv.DictWriter(output_handle, csv_fieldnames)
        csv_writer.writeheader()
        for record, pred, pred_std in zip(csv_records, list(y_pred), list(y_pred_std)):
            record.update({'dpImmun': pred,
                           'dpImmun_std': pred_std,
                           'dpImmun_neg_pct': np.sum(pred >= model_y_pred_0) / len(model_y_pred_0),
                           'dpImmun_pos_pct': np.sum(pred >= model_y_pred_1) / len(model_y_pred_1),
                           'dpImmun_10k_pct': np.sum(pred >= y_random) / len(y_random),
                          })
            csv_writer.writerow(record)

def predict_rescore(model, input_file, output_file):
    print("predict()")
    print("input_file = ", input_file)
    print("output_file = ", output_file)
    peptide_list = []
    response_list = []
    score_list = []
    rank_list = []
    with open(input_file, 'r') as input_handle:
        csv_reader = csv.DictReader(input_handle, delimiter=',')
        for row in csv_reader:
            peptide = row['peptide']
            response = row['response']
            score = float(row['Ave'])
            rank = float(row['Rank'])
            peptide_list.append(peptide)
            response_list.append(response)
            score_list.append(score)
            rank_list.append(rank)
    y_pred = predict_sub(model, peptide_list)
    y_rescore = np.sqrt(y_pred * np.array(score_list))
    with open(output_file, 'w') as output_handle:
        headers = ['peptide','response', 'Ave', 'Rank', 'dpImmun', 'dpImmun_rescore']
        csv_writer = csv.DictWriter(output_handle, fieldnames=headers)
        csv_writer.writeheader()
        for peptide, response, score, rank, pred, rescore in zip(
            peptide_list, response_list, score_list, rank_list, list(y_pred), list(y_rescore)):
            csv_writer.writerow({'peptide': peptide,
                                 'response': response,
                                 'Ave': score,
                                 'Rank': rank,
                                 'dpImmun': pred,
                                 'dpImmun_rescore': rescore})


In [ ]:
# [need input] patient prediction
print("patient prediction")
input_file = "test." + patient + ".csv" # "test.mel15.csv
output_file = input_file + ".dpimmun.csv"
# input_file = "deepnovo_peptides.csv.sequence.txt"
# output_file = "deepnovo_peptides.csv.dpimmun.csv"
print("input_file = ", input_file)
print("output_file = ", output_file)

model_number = 100
model_paths = [model_name + "/" + "model" + "_" + model_name + "_" + str(m) + ".h5" for m in range(model_number)]
model_score_path = model_name + "/" + "model_score" + ".pkl"
print("len(model_paths) =", len(model_paths))
print("model_paths[0] =", model_paths[0])
print("model_score_path =", model_score_path)
print()

# use average of the best models for predictions
best = 10
print(best, "best models")
print()
model_list = [load_model(model_paths[i]) for i in sorted_auc_indices[:best]]
with open(model_score_path, 'rb') as file:
    model_score = pickle.load(file) # y_testing, y_pred_list
y_testing, y_pred_list = model_score
y_pred_avg = np.mean([y_pred_list[i] for i in sorted_auc_indices[:best]], axis=0)
predict(model_list, [y_testing, y_pred_avg], input_file, output_file)


In [ ]:
##########################################################
##########################################################
##########################################################
# EVALUATION
##########################################################
##########################################################
##########################################################

In [ ]:
# evaluation
def evaluate_sub(y_test, y_score, threshold):
#     y_pred = (y_score >= threshold).astype(int)
#     y_true = (y_pred == y_test).astype(int)
#     y_true_pos = y_true * y_test
#     y_true_neg = y_true * (1 - y_test)
#     pred_sensitivity = np.sum(y_true_pos) / np.sum(y_test)
#     pred_specifity = np.sum(y_true_neg) / np.sum(1 - y_test)
#     pred_precision = np.sum(y_true_pos) / np.sum(y_pred)
#     pred_precision_neg = np.sum(y_true_neg) / np.sum(1 - y_pred)
#     pred_accuracy = np.sum(y_true) / len(y_true)
#     pred_accuracy_balanced = (pred_sensitivity + pred_specifity) / 2
#     pred_auc = roc_auc_score(y_test, y_score)
#     pred_f1 = 1 / ((1/pred_sensitivity + 1/pred_precision) / 2)
    
#     sorted_index = np.argsort(-y_score)
#     y_test_sorted = y_test[sorted_index]
#     top20 = y_test_sorted[:20]
#     top20_pos = np.sum(top20)
#     top20_precision = top20_pos / 20
#     top20_sensitivity = top20_pos / np.sum(y_test)
#     pos_rank = [index+1 for index, val in enumerate(y_test_sorted) if val == 1]
#     pos_rank_median = np.median(pos_rank)
    
#     print("pred_sensitivity = ", pred_sensitivity)
#     print("pred_specifity = ", pred_specifity)
#     print("pred_precision = ", pred_precision)
#     print("pred_precision_neg = ", pred_precision_neg)
#     print("pred_accuracy = ", pred_accuracy)
#     print("pred_accuracy_balanced = ", pred_accuracy_balanced)
#     print("pred_auc = ", pred_auc)
#     print("pred_f1 = ", pred_f1)
#     print("top20_pos = ", top20_pos)
#     print("top20_precision = ", top20_precision)
#     print("top20_sensitivity = ", top20_sensitivity)
#     print("pos_rank_median = ", pos_rank_median)
#     print()
    
    auc  = roc_auc_score(y_test, y_score)
    return auc

def evaluate(input_file, tool_name):
    y_test = []
    y_score = []
    with open(input_file, 'r') as input_handle:
        csv_reader = csv.DictReader(input_handle, delimiter=',')
        for row in csv_reader:
            peptide = row['peptide']
            response = row['response']
            assert response == "positive" or response == "negative"
            response = 1 if response == "positive" else 0
            y_test.append(response)
            score = float(row[tool_name])
            if "%rank" in tool_name:
                score = 1 - score
            y_score.append(score)
    y_test = np.array(y_test)
    y_score = np.array(y_score)

    threshold =0.0
    if tool_name == 'PRIME':
        threshold = 0.0
    elif tool_name == 'NetMHCpan':
        threshold = 0.0
    elif tool_name == 'IEDB':
        threshold = 0.0
    elif tool_name == 'dpImmun':
        threshold = 0.5
    elif tool_name == 'Rank':
        y_score = -y_score # for NetMHCpan rank, lower is better
        threshold = -2.0
    elif tool_name == 'dpImmun_rescore':
        threshold = 0.5
    
    return y_test, y_score, evaluate_sub(y_test, y_score, threshold)


In [ ]:
# [need input] patient auc evaluation
eval_file = "test." + patient + ".csv.dpimmun.csv"
print("auc evaluation")
print("eval_file =", eval_file)
print()
tools = ['dpImmun', 'PRIME %rank', 'NetMHCpan %rank', 'IEDB']
tools_auc = [evaluate(eval_file, tool) for tool in tools]
names = ['dpImmun', 'PRIME', 'NetMHCpan', 'IEDB']
colors = ['red', 'orange', 'green', 'blue']

pyplot.title('Receiver Operating Characteristic Curves')
for y, n, c in zip(tools_auc, names, colors):
#     print(n, y[2], sep='\t')
    fpr, tpr, threshold = roc_curve(y[0], y[1])
    pyplot.plot(fpr, tpr, color=c, label="{0:s}, AUC={1:0.2f}".format(n, y[2]))
pyplot.legend(loc = 'lower right')
pyplot.xlim([0, 1])
pyplot.ylim([0, 1.05])
pyplot.ylabel('True Positive Rate')
pyplot.xlabel('False Positive Rate')
#     pyplot.show()
print()
